# MRO: LAB 1 - Wymiarowość danych
# Piotr Rząsa

In [1]:
import numpy as np

from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d import proj3d
from random import randint
from sklearn.decomposition import PCA as sklearnPCA
from sklearn.neighbors.dist_metrics import DistanceMetric, MinkowskiDistance
from sklearn.metrics.pairwise import cosine_distances, distance_metrics, paired_cosine_distances

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
def create_data(wymiar):
    LICZBA = 100
    WYMIAR = wymiar
    L_KLAS = 4
    ZABURZENIE = 3

    for i in range(L_KLAS):

        # mu_vec - współrzędne wokół których powinny się rozmieścić punkty
        # cov_mat - jak bardzo powinny być rozrzucone względem każdego wymiaru po kolei

        mu_vec2 = np.ones((WYMIAR,), dtype=np.int)
        mu_vec2 = np.multiply(mu_vec2, i*2)

        cov_mat2 = np.zeros((WYMIAR, WYMIAR), int)
        np.fill_diagonal(cov_mat2, 1)
        zabu = np.random.randint(ZABURZENIE, size=(WYMIAR, 1))
        cov_mat2 = np.multiply(cov_mat2, np.add(zabu,1))

        class2_sample = np.random.multivariate_normal(mu_vec2, cov_mat2, LICZBA).T
        assert class2_sample.shape == (WYMIAR,LICZBA), "The matrix has not the right dimensions"

        # konkatenacja danych
        if(i == 0):
            all_samples = class2_sample
        else:
            all_samples = np.concatenate((all_samples, class2_sample), axis=1)

    #print(all_samples.shape)
    assert all_samples.shape == (WYMIAR,LICZBA*L_KLAS), "The output matrix has not the right dimensions"

    return all_samples.T

In [4]:
# wyliczenie odległości miedzy punktami
def gen_dim(data, wymiar, metric, P=0):

    if (metric == "cosinus"):
        original_dist = cosine_distances(data)
    elif P==0:
        dist = DistanceMetric.get_metric(metric)
        original_dist = dist.pairwise(data)
    else:
        dist = DistanceMetric.get_metric(metric,p=P)
        original_dist = dist.pairwise(data)
        
    original_dist.sort()
    mean_dist = original_dist.mean()
    mean_closest = original_dist[:,1].mean()
    mean_two_closest = original_dist[:,1:3].mean()

    if (P!=0):
        print("\nLiczba wymiarów: {}, metryka: {} p={}".format(wymiar, metric, P))
    else:
        print("\nLiczba wymiarów: {}, metryka: {}".format(wymiar, metric))

    print("\t%0.2f - średnia odległość do innych punktów" % mean_dist)
    print("\t%0.2f - średnia odległość najbliższego punktu" % mean_closest)
    print("\t%0.2f - średnia odległość dwóch najbliższych punktów" % mean_two_closest)

## Generacja / adaptacja danych:

#### Własne, generowane dane:
- 4 klasy
- 400 elementów zbioru 
- zestaw danych o 10 wymiarach
- zestaw danych o 2000 wymiarów

#### Część zbioru MNIST:
- 10 klas
- 10 000 elementów zbioru
- 784 wymiary

In [5]:
data10 = create_data(10)
data2000 = create_data(2000)

### Liczenie odległości punktów dla własnego zbioru:

In [6]:
gen_dim(data10, 10, 'cosinus')
gen_dim(data2000, 2000, 'cosinus')
gen_dim(data10, 10, 'chebyshev')
gen_dim(data2000, 2000, 'chebyshev')
gen_dim(data10, 10, 'minkowski', 1)
gen_dim(data2000, 2000, 'minkowski', 1)
gen_dim(data10, 10, 'minkowski', 2)
gen_dim(data2000, 2000, 'minkowski', 2)
gen_dim(data10, 10, 'minkowski', 4)
gen_dim(data2000, 2000, 'minkowski', 4)


Liczba wymiarów: 10, metryka: cosinus
	0.53 - średnia odległość do innych punktów
	0.09 - średnia odległość najbliższego punktu
	0.10 - średnia odległość dwóch najbliższych punktów

Liczba wymiarów: 2000, metryka: cosinus
	0.53 - średnia odległość do innych punktów
	0.31 - średnia odległość najbliższego punktu
	0.32 - średnia odległość dwóch najbliższych punktów

Liczba wymiarów: 10, metryka: chebyshev
	5.79 - średnia odległość do innych punktów
	1.81 - średnia odległość najbliższego punktu
	1.92 - średnia odległość dwóch najbliższych punktów

Liczba wymiarów: 2000, metryka: chebyshev
	10.03 - średnia odległość do innych punktów
	6.64 - średnia odległość najbliższego punktu
	6.72 - średnia odległość dwóch najbliższych punktów

Liczba wymiarów: 10, metryka: minkowski p=1
	30.15 - średnia odległość do innych punktów
	8.05 - średnia odległość najbliższego punktu
	8.52 - średnia odległość dwóch najbliższych punktów

Liczba wymiarów: 2000, metryka: minkowski p=1
	6036.99 - średnia odległoś

### Liczenie odległości punktów dla MNIST:

In [ ]:
mn = mnist.test.images

In [8]:
gen_dim(mn, mn.shape[1], 'cosinus')
gen_dim(mn, mn.shape[1], 'chebyshev')
gen_dim(mn, mn.shape[1], 'minkowski', 1)
gen_dim(mn, mn.shape[1], 'minkowski', 2)
gen_dim(mn, mn.shape[1], 'minkowski', 4)


Liczba wymiarów: 784, metryka: cosinus
	0.59 - średnia odległość do innych punktów
	0.12 - średnia odległość najbliższego punktu
	0.13 - średnia odległość dwóch najbliższych punktów

Liczba wymiarów: 784, metryka: chebyshev
	1.00 - średnia odległość do innych punktów
	0.93 - średnia odległość najbliższego punktu
	0.94 - średnia odległość dwóch najbliższych punktów

Liczba wymiarów: 784, metryka: minkowski p=1
	130.82 - średnia odległość do innych punktów
	41.91 - średnia odległość najbliższego punktu
	43.81 - średnia odległość dwóch najbliższych punktów

Liczba wymiarów: 784, metryka: minkowski p=2
	10.18 - średnia odległość do innych punktów
	4.62 - średnia odległość najbliższego punktu
	4.78 - średnia odległość dwóch najbliższych punktów


KeyboardInterrupt: 

#### Komantarz do punktu 5:
Najbardziej odporna na przekleństwo wymiarowości jest odległość cosinusowa. Jest tak ponieważ jest ona z zakresu 1 +/- 1. Dzięki temu nawe dla ogromnego wzrostu odelgłości wielowymiarowej odległość cosinusowa pozostaje w stałym przedziale.<br>
Najgorsze rezultaty uzyskałem dla metryki minkowskiego dla p=1.

#### Komantarz do punktu 6:
Binaryzacja odległości moim zdaniem bardzo dobrze przeciwdziała przekleństwu wymiarowości, ponieważ dany element w stosunku do innego może się znajdować tylko w dwóch stanach - w jego otoczeniu / blisko lub daleko. 

## Lab 2 - PCA - tworzyłem referat.